In [ ]:
import pydicom
import matplotlib.pyplot as plt
import numpy as np

def print_stats(rois, rd):
    header = "{:^16} {:^9} {:^9} {:^9}".format('Region', 'Min', 'Avg', 'Max')
    print('-'*len(header))
    print(header)
    print('-'*len(header))
    for seq in rd.DVHSequence:
        name = rois[seq.DVHReferencedROISequence[0].ReferencedROINumber]
        print("{:<16} {:>9.4f} {:>9.4f} {:>9.4f}".format(name, seq.DVHMinimumDose, seq.DVHMeanDose, seq.DVHMaximumDose))

def plot_dvh(rois, rd):
    plt.style.use("default")
    plt.rcParams['figure.figsize'] = [12, 8]
    plt.rcParams['font.size'] = 18

    for seq in rd.DVHSequence:
        data = seq.DVHData[1::2]
        rel_data = data
        if seq.DVHVolumeUnits == 'CM3':
            rel_data = [x*100/data[0] for x in data]
        label = rois[seq.DVHReferencedROISequence[0].ReferencedROINumber]
        plt.plot(np.arange(0, len(rel_data))/100, rel_data, linewidth=3, label=label)

    plt.legend(loc='upper right')
    plt.grid()
    plt.ylabel("Volume (%)")
    plt.xlabel("Dose (Gy)")
    plt.xticks(np.arange(0,105,5))
    plt.xlim((-5, 105))
    plt.yticks(np.arange(0,105,10))
    plt.tight_layout()
    plt.show()

In [ ]:
rs_filename = '../DICOM/16/RS.1.2.246.352.205.5438954218386416801.8699273283688404648.dcm'
rd_filename = '../DICOM/16/plan_eud_20210409/RD.1.2.246.352.221.52316248413044920758968839694241191817.dcm'

rs = pydicom.dcmread(rs_filename)
rd = pydicom.dcmread(rd_filename)

rois = {}
for seq in rs.RTROIObservationsSequence:
    rois[seq.ReferencedROINumber] = seq.ROIObservationLabel

print_stats(rois, rd)
plot_dvh(rois, rd)